In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('D:\Code\ML\Machine_Learning\Data\Hotel_Reviews.csv')
df.info()

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\TheAssshOne\AppData\Local\Temp\ipykernel_18692\1020553786.py:1: SyntaxWarning: invalid escape sequence '\C'
  df = pd.read_csv('D:\Code\ML\Machine_Learning\Data\Hotel_Reviews.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

### Clean the data a bit more

initial column processing

In [3]:
# dropping columns we will not use:
df.drop(["lat", "lng"], axis = 1, inplace=True)

In [4]:
def replace_address(row):
    if "Netherlands" in row["Hotel_Address"]:
        return "Amsterdam, Netherlands"
    elif "Barcelona" in row["Hotel_Address"]:
        return "Barcelona, Spain"
    elif "United Kingdom" in row["Hotel_Address"]:
        return "London, United Kingdom"
    elif "Milan" in row["Hotel_Address"]:        
        return "Milan, Italy"
    elif "France" in row["Hotel_Address"]:
        return "Paris, France"
    elif "Vienna" in row["Hotel_Address"]:
        return "Vienna, Austria" 

# Replace all the addresses with a shortened, more useful form
df["Hotel_Address"] = df.apply(replace_address, axis = 1)
# The sum of the value_counts() should add up to the total number of reviews
print(df["Hotel_Address"].value_counts())

Hotel_Address
London, United Kingdom    262301
Barcelona, Spain           60149
Paris, France              59928
Amsterdam, Netherlands     57214
Vienna, Austria            38939
Milan, Italy               37207
Name: count, dtype: int64


query country level data:

In [5]:
display(df.groupby("Hotel_Address").agg({"Hotel_Name": "nunique"}))

,Hotel_Name
Hotel_Address,
"Amsterdam, Netherlands",105
"Barcelona, Spain",211
"London, United Kingdom",400
"Milan, Italy",162
"Paris, France",458
"Vienna, Austria",158


In [6]:
# Drop `Additional_Number_of_Scoring`
df.drop("Additional_Number_of_Scoring", axis = 1, inplace=True)

# Replace Total_Number_of_Reviews with the total number of reviews for that hotel that are actually in the dataset
df["Total_Number_of_Reviews"] = df.groupby("Hotel_Name")["Total_Number_of_Reviews"].transform("count")

# Replace Average_Score with our own calculated score
df["Average_Score"] = df.groupby("Hotel_Name")["Reviewer_Score"].transform("mean")

process review columns

In [7]:
# rop Review_Total_Negative_Word_Counts, Review_Total_Positive_Word_Counts, Review_Date and days_since_review, Total_Number_of_Reviews_Reviewer_Has_Given
df.drop(["Review_Total_Negative_Word_Counts", "Review_Total_Positive_Word_Counts", "Review_Date", "days_since_review", "Total_Number_of_Reviews_Reviewer_Has_Given"], axis = 1, inplace=True)

process tag columns

In [8]:
# Remove opening and closing brackets
df.Tags = df.Tags.str.strip("[]")
# Remove all quotes
df.Tags = df.Tags.str.replace(" ', '", ",", regex = False)

In [9]:
# Process the Tags column
df["Leisure_trip"] = df.Tags.apply(lambda tag: 1 if "Leisure trip" in tag else 0)
df["Couple"] = df.Tags.apply(lambda tag: 1 if "Couple" in tag else 0)
df["Solo_traveler"] = df.Tags.apply(lambda tag: 1 if "Solo traveler" in tag else 0)
df["Business_trip"] = df.Tags.apply(lambda tag: 1 if "Business trip" in tag else 0)
df["Group"] = df.Tags.apply(lambda tag: 1 if "Group" in tag or "Travelers with friends" in tag else 0)
df["Family_with_young_children"] = df.Tags.apply(lambda tag: 1 if "Family with young children" in tag else 0)
df["Family_with_older_children"] = df.Tags.apply(lambda tag: 1 if "Family with older children" in tag else 0)
df["With_a_pet"] = df.Tags.apply(lambda tag: 1 if "With a pet" in tag else 0)

### Save the file

In [11]:
print("Saving results to Hotel_Reviews_Filtered.csv")
df.to_csv("D:\Code\ML\Machine_Learning\Data\Hotel_Reviews_Filtered.csv", index = False)

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\TheAssshOne\AppData\Local\Temp\ipykernel_18692\3144016386.py:2: SyntaxWarning: invalid escape sequence '\C'
  df.to_csv("D:\Code\ML\Machine_Learning\Data\Hotel_Reviews_Filtered.csv", index = False)


Saving results to Hotel_Reviews_Filtered.csv
